In [15]:
%load_ext autoreload
%autoreload 2

from setup_imports import *  # noqa: F401,F403
from src.phrases.phrase_model import Phrase, Translation, get_phrase, get_phrase_by_english
from src.models import BCP47Language
from src.connections.gcloud_auth import get_firestore_client
from src.audio import get_voice_model, generate_translation_audio
from src.images import generate_image
from PIL import Image

# Hello World Test - Full Workflow
# =================================

OUR_PHRASE = "Hello, world!"
phrase = Phrase.create_phrase(OUR_PHRASE, source="manual")
print(f"✓ Created phrase: {phrase.phrase_hash}")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
✓ Created phrase: hello_world_315f5b


In [18]:
phrase.translations

[Translation(phrase_hash='hello_world_315f5b', language=Language.make(language='en', territory='GB'), text='Hello, world!', text_lower='hello, world!', tokens=['hello', 'world'], audio=None, image_file_path=None, audio_segment=None, image=None)]

In [19]:

# Add French translation
fr = phrase.translate(BCP47Language.get("fr-FR"), refine=False)
print(f"✓ French: {fr.text}")


✓ French: Bonjour le monde!


In [20]:

# Add German translation
de = phrase.translate(BCP47Language.get("de-DE"), refine=True)
print(f"✓ German: {de.text}")


✓ German: Hallo, Welt!


In [ ]:
# Generate audio for each translation using the new generate_audio() method
# For flashcard context: generates slow and normal speeds
# For story context: generates normal and fast speeds

# French audio
print("Generating French audio (flashcard)...")
fr.generate_audio(context="flashcard", bucket_name="audio-language-trainer-private-content")
print(f"✓ French audio generated: {len(fr.audio)} audio files")
for audio in fr.audio:
    print(f"  - {audio.context} / {audio.speed}: {audio.file_path}")

# German audio
print("\nGenerating German audio (flashcard)...")
de.generate_audio(context="flashcard", bucket_name="audio-language-trainer-private-content")
print(f"✓ German audio generated: {len(de.audio)} audio files")
for audio in de.audio:
    print(f"  - {audio.context} / {audio.speed}: {audio.file_path}")

In [ ]:
# Get the English translation and generate image
en_gb = phrase.translations[0]  # The en-GB translation created in create_phrase()
print(f"✓ English translation: {en_gb.text}")

# Generate image for the English translation (shared across all language variants)
image_prompt = "A cheerful greeting scene with two people waving hello, bright morning sunlight"
image = generate_image(image_prompt, style="ghibli", project_id="swedish-course")
en_gb.image = image
en_gb.upload_image("audio-language-trainer-private-content")
print(f"✓ Image generated and attached: {en_gb.image_file_path}")

In [ ]:
# Demonstrate the new audio structure and batch upload
print("Audio structure for French translation:")
print(f"  Total audio files: {len(fr.audio)}")
for audio in fr.audio:
    print(f"  - Context: {audio.context}, Speed: {audio.speed}, Provider: {audio.voice_provider}, Duration: {audio.duration_seconds}s")

print("\nAudio structure for German translation:")
print(f"  Total audio files: {len(de.audio)}")
for audio in de.audio:
    print(f"  - Context: {audio.context}, Speed: {audio.speed}, Provider: {audio.voice_provider}, Duration: {audio.duration_seconds}s")

# Test batch upload all audio from the phrase to GCS
print("\nUploading all audio from phrase to GCS...")
phrase.upload_all_audio(bucket_name="audio-language-trainer-private-content")
print("✓ All audio uploaded successfully")

In [ ]:
# Test story context audio generation (generates normal and fast speeds)
print("Generating French audio (story context)...")
fr.generate_audio(context="story", bucket_name="audio-language-trainer-private-content")
print(f"✓ French story audio generated: {len(fr.audio)} total audio files")
print("  Audio speeds by context:")
flashcard_audio = [a for a in fr.audio if a.context == "flashcard"]
story_audio = [a for a in fr.audio if a.context == "story"]
print(f"  - Flashcard: {sorted([a.speed for a in flashcard_audio])}")
print(f"  - Story: {sorted([a.speed for a in story_audio])}")

# Verify the speeds are correct
assert all(a.speed in ["slow", "normal"] for a in flashcard_audio), "Flashcard should have slow and normal"
assert all(a.speed in ["normal", "fast"] for a in story_audio), "Story should have normal and fast"
print("✓ Audio speeds are correct for each context")

In [ ]:
# Upload complete phrase to Firestore with all translations
db = get_firestore_client("firephrases")
phrase_hash = phrase.upload_phrase(db)
print(f"✓ Phrase uploaded to Firestore: {phrase_hash}")


In [ ]:
# Retrieve complete phrase from Firestore
retrieved_phrase = get_phrase(phrase_hash)
print(f"✓ Retrieved from Firestore: {retrieved_phrase.english}")
print(f"✓ Translations: {', '.join([t.language.to_tag() for t in retrieved_phrase.translations])}")

# Show audio metadata for retrieved translations
print("\nAudio metadata retrieved from Firestore:")
for translation in retrieved_phrase.translations:
    if translation.audio:
        print(f"  {translation.language.to_tag()}: {len(translation.audio)} audio files")
        for audio in translation.audio:
            print(f"    - {audio.context} / {audio.speed}: {audio.duration_seconds}s ({audio.voice_provider})")
    else:
        print(f"  {translation.language.to_tag()}: No audio")

# Phrase object is ready for inspection - pull audio/images as needed
retrieved_phrase